In [121]:
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [122]:
base_path = 'data/nasa'
starting_year = 2020
ending_year = 2025

In [123]:
all_subdirs = [
    d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))
]

year_dirs = []
for d in all_subdirs:
    if d.isdigit():
        year_int = int(d)
        if starting_year <= year_int <= ending_year:
            year_dirs.append(year_int)
year_dirs.sort()

In [124]:
merged_dir = os.path.join(base_path, 'merged')
os.makedirs(merged_dir, exist_ok=True)

In [125]:
for var_num in range(1, 36):
    dfs = []

    for year in year_dirs:
        filename = f"POWER_Regional_Daily_{year}0101_{year}1231 ({var_num}).csv"
        file_path = os.path.join(base_path, str(year), filename)

        if os.path.exists(file_path):
            df = pd.read_csv(file_path, skiprows=9)
            dfs.append(df)

    if dfs:
        merged_df = pd.concat(dfs, ignore_index=True)
        merged_df.sort_values(by="LAT", inplace=True)
        merged_df.reset_index(drop=True, inplace=True)

        out_filename = f"POWER_Regional_Daily_Merged ({var_num}).csv"
        out_path = os.path.join(merged_dir, out_filename)
        merged_df.to_csv(out_path, index=False)

        print(f"Variable ({var_num}) merged and saved to {out_path}")
    else:
        print(f"No files found for variable ({var_num}) in the given year range.")

Variable (1) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (1).csv
Variable (2) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (2).csv
Variable (3) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (3).csv
Variable (4) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (4).csv
Variable (5) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (5).csv
Variable (6) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (6).csv
Variable (7) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (7).csv
Variable (8) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (8).csv
Variable (9) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (9).csv
Variable (10) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (10).csv
Variable (11) merged and saved to data/nasa\merged\POWER_Regional_Daily_Merged (11).csv
Variable (12) merged and saved to data/nasa\merged

In [126]:
def merge_all_variables(
    merged_dir="data/nasa/merged", output_file="all_variables_merged.csv"
):
    """
    Merges all CSV files in `merged_dir` that match the pattern:
    'POWER_Regional_Daily_Merged (*.csv)'.

    Each CSV is expected to have:
        LAT, LON, YEAR, MO, DY, <VARIABLE_COLUMN>
    The script:
        1. Reads each CSV.
        2. Identifies the variable column (anything not in {LAT,LON,YEAR,MO,DY}).
        3. Performs an outer merge on [LAT, LON, YEAR, MO, DY].
        4. Sorts by these key columns and writes the final DataFrame to `output_file`.
    """

    key_cols = ["LAT", "LON", "YEAR", "MO", "DY"]

    all_files = [
        f
        for f in os.listdir(merged_dir)
        if f.startswith("POWER_Regional_Daily_Merged") and f.endswith(".csv")
    ]

    all_files.sort()

    merged_df = None

    for csv_file in all_files:
        file_path = os.path.join(merged_dir, csv_file)

        df = pd.read_csv(file_path)

        var_cols = [c for c in df.columns if c not in key_cols]

        if len(var_cols) == 1:
            var_name = var_cols[0]

            if merged_df is None:

                merged_df = df
            else:

                merged_df = pd.merge(merged_df, df, on=key_cols, how="outer")
        else:
            print(
                f"Warning: {csv_file} has {len(var_cols)} variable columns; skipping."
            )

    if merged_df is not None:
        merged_df.sort_values(by=key_cols, inplace=True)
        merged_df.reset_index(drop=True, inplace=True)

        output_path = os.path.join(merged_dir, output_file)
        merged_df.to_csv(output_path, index=False)
        print(f"All variables merged. Final file saved at: {output_path}")
    else:
        print("No valid files found to merge or no variable columns detected.")


merge_all_variables()

All variables merged. Final file saved at: data/nasa/merged\all_variables_merged.csv


In [127]:
nasa_data = pd.read_csv("data/nasa/merged/all_variables_merged.csv")
nasa_data.head()

,LAT,LON,YEAR,MO,DY,CLRSKY_SFC_SW_DWN_x,ALLSKY_SFC_UV_INDEX,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN_y,WS2M,...,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB
0,29.5,34.0,2024,1,1,-999.0,NaN,-999.0,-999.0,NaN,...,NaN,NaN,NaN,NaN,NaN,-999.0,NaN,NaN,NaN,NaN
1,29.5,34.0,2024,1,2,-999.0,NaN,-999.0,-999.0,NaN,...,NaN,NaN,NaN,NaN,NaN,-999.0,NaN,NaN,NaN,NaN
2,29.5,34.0,2024,1,3,-999.0,NaN,-999.0,-999.0,NaN,...,NaN,NaN,NaN,NaN,NaN,-999.0,NaN,NaN,NaN,NaN
3,29.5,34.0,2024,1,4,-999.0,NaN,-999.0,-999.0,NaN,...,NaN,NaN,NaN,NaN,NaN,-999.0,NaN,NaN,NaN,NaN
4,29.5,34.0,2024,1,5,-999.0,NaN,-999.0,-999.0,NaN,...,NaN,NaN,NaN,NaN,NaN,-999.0,NaN,NaN,NaN,NaN


In [128]:
missing_values_before = nasa_data.isnull().sum()

missing_data_summary_before = pd.DataFrame({
    "Missing Values": missing_values_before,
    "Percentage": (missing_values_before / len(nasa_data)) * 100
}).sort_values(by="Missing Values", ascending=False)

missing_data_summary_before.head()

,Missing Values,Percentage
ALLSKY_SFC_UV_INDEX,52632,78.265525
ALLSKY_SFC_SW_DIFF,52632,78.265525
ALLSKY_SFC_UVB,52632,78.265525
ALLSKY_SFC_PAR_TOT,52632,78.265525
ALLSKY_KT,52632,78.265525


In [129]:
nasa_data.sort_values(by=["LAT", "LON", "YEAR", "MO", "DY"], inplace=True)


# nasa_data.interpolate(method="linear", limit_direction="both", inplace=True)


missing_values_after = nasa_data.isnull().sum()


missing_data_summary_after = pd.DataFrame(
    {
        "Missing Values": missing_values_after,
        "Percentage": (missing_values_after / len(nasa_data)) * 100,
    }
).sort_values(by="Missing Values", ascending=False)

missing_data_summary_after.head()

,Missing Values,Percentage
ALLSKY_SFC_UV_INDEX,52632,78.265525
ALLSKY_SFC_SW_DIFF,52632,78.265525
ALLSKY_SFC_UVB,52632,78.265525
ALLSKY_SFC_PAR_TOT,52632,78.265525
ALLSKY_KT,52632,78.265525


In [130]:
# nasa_data.to_csv("data/nasa/merged/all_variables_merged_interpolated.csv", index=False)
# nasa_data.shape

In [131]:
# nasa_data.info()

I'll start by inspecting the dataset to understand its structure and completeness. Then, I'll prepare it for the **Renewable Energy Consumption Tracker** by applying necessary data cleaning, feature engineering, and transformations. Let me analyze the dataset first.

# Key Observations:
1. **Missing Data Representation:** The dataset uses `-999` as a placeholder for missing values instead of `NaN`. These need to be replaced for proper handling.

2. **Duplicate Columns:** `CLRSKY_SFC_SW_DWN_x` and `CLRSKY_SFC_SW_DWN_y` appear to be duplicate variables.

3. **Latitude and Longitude Range Validation:** Some latitude (LAT) and longitude (LON) values (e.g., 29.5°N, 34.0°E) are outside Palestine’s expected range (31°N-33°N, 34°E-36°E), requiring filtering.

4. **Outlier Detection Needed:** Some columns may contain extreme values beyond physically reasonable limits.

5. **Key Variables for Renewable Energy:**
- **Solar Energy Indicators:** `ALLSKY_SFC_SW_DWN`, `CLRSKY_SFC_SW_DWN`, `ALLSKY_SFC_SW_DNI`, `ALLSKY_SFC_UV_INDEX`, `ALLSKY_SFC_PAR_TOT`, `CLRSKY_SFC_PAR_TOT`
- **Wind Energy Indicators:** `WS10M`, `WS10M_MAX`, `WS50M`, `WS50M_MAX`
- **Weather Factors:** `T2M (Temperature)`, `RH2M (Humidity)`, `PRECTOTCORR (Precipitation)`

# Next Steps in Data Preparation:

- Replace `-999` values with `NaN` and handle missing values.

- Remove duplicate and unnecessary columns.

- Filter dataset to keep only valid LAT/LON values.

- Detect and handle outliers using Z-score filtering.

- Normalize/scale the relevant features for better model performance.


In [132]:
nasa_data_copy = nasa_data.copy()

In [133]:
nasa_data.replace(-999.0, np.nan, inplace=True)

# calculate the sum of missing values in each row
# nasa_data["missing_values"] = nasa_data.isnull().sum(axis=1)
# nasa_data["missing_values"]
# nasa_data.to_csv('outputs/exploring_outputs/nasa/missing_values.csv', index=False)
# nasa_data.dropna(inplace=True)

# show which columns have missing values
nasa_data.isnull().sum()
missing_cols = nasa_data.columns[nasa_data.isnull().any()].tolist()
missing_cols

# nasa_data.shape

['CLRSKY_SFC_SW_DWN_x',
 'ALLSKY_SFC_UV_INDEX',
 'ALLSKY_SFC_SW_DWN',
 'CLRSKY_SFC_SW_DWN_y',
 'WS2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'TS',
 'T2M_RANGE',
 'T2M_MAX',
 'ALLSKY_SFC_SW_DNI',
 'T2M_MIN',
 'QV2M',
 'RH2M',
 'PRECTOTCORR',
 'PS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WD10M',
 'ALLSKY_SFC_SW_DIFF',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE',
 'WD50M',
 'ALLSKY_KT',
 'ALLSKY_SFC_LW_DWN',
 'ALLSKY_SFC_PAR_TOT',
 'CLRSKY_SFC_PAR_TOT',
 'ALLSKY_SFC_UVA',
 'ALLSKY_SFC_UVB']

In [134]:
nasa_with_missing = nasa_data[missing_cols]
nasa_with_missing.describe()

,CLRSKY_SFC_SW_DWN_x,ALLSKY_SFC_UV_INDEX,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN_y,WS2M,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,...,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB
count,14503.000000,13384.000000,15220.000000,14503.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,43848.000000,...,43848.000000,43848.000000,43848.000000,43848.000000,13384.000000,15232.000000,13384.000000,13384.000000,13384.000000,13384.000000
mean,6.259642,1.752719,5.778076,6.259642,2.589530,20.435735,10.546738,15.491251,21.577271,11.624326,...,7.342253,2.198981,5.143272,239.063839,0.657672,8.087575,2.619261,2.787572,0.333486,0.007219
std,1.734811,0.906095,1.979614,1.734811,1.212567,6.701274,6.259571,5.850757,7.473359,5.535678,...,2.006157,1.603866,1.800465,109.772648,0.108180,0.860192,0.888182,0.780000,0.117923,0.004992
min,2.640000,0.140000,0.700000,2.640000,0.280000,0.260000,-12.300000,-5.270000,-0.200000,0.150000,...,1.300000,0.000000,0.690000,0.000000,0.130000,5.330000,0.380000,1.140000,0.050000,0.000000
25%,4.610000,0.870000,4.030000,4.610000,1.820000,15.060000,6.200000,11.010000,15.370000,8.120000,...,5.940000,0.990000,3.870000,164.275000,0.630000,7.470000,1.820000,2.040000,0.220000,0.000000
50%,6.480000,1.730000,5.970000,6.480000,2.430000,21.150000,10.410000,15.810000,22.230000,12.750000,...,7.140000,1.870000,4.990000,283.600000,0.690000,8.160000,2.730000,2.900000,0.340000,0.010000
75%,7.900000,2.560000,7.600000,7.900000,3.170000,26.300000,14.940000,20.220000,28.140000,15.770000,...,8.410000,3.020000,6.220000,322.200000,0.730000,8.740000,3.460000,3.540000,0.450000,0.010000
max,9.120000,3.860000,9.090000,9.120000,12.250000,37.460000,28.070000,28.480000,38.910000,25.700000,...,21.120000,15.630000,19.960000,360.000000,0.810000,10.770000,3.950000,3.950000,0.530000,0.020000


In [135]:
nasa_data.drop(columns=["CLRSKY_SFC_SW_DWN_x", "CLRSKY_SFC_SW_DWN_y"], inplace=True)

In [136]:
palestine_lat_range = (31, 33)
palestine_lon_range = (34, 36)

nasa_data = nasa_data[
    (nasa_data["LAT"] >= palestine_lat_range[0]) & (nasa_data["LAT"] <= palestine_lat_range[1]) &
    (nasa_data["LON"] >= palestine_lon_range[0]) & (nasa_data["LON"] <= palestine_lon_range[1])
]

In [137]:
missing_after_filtering = nasa_data.isnull().sum()
missing_after_filtering[missing_after_filtering > 0]

ALLSKY_SFC_UV_INDEX    33877
ALLSKY_SFC_SW_DWN      32500
WS2M                   13164
T2M                    13164
T2MDEW                 13164
T2MWET                 13164
TS                     13164
T2M_RANGE              13164
T2M_MAX                13164
ALLSKY_SFC_SW_DNI      33877
T2M_MIN                13164
QV2M                   13164
RH2M                   13164
PRECTOTCORR            13164
PS                     13164
WS10M                  13164
WS10M_MAX              13164
WS10M_MIN              13164
WS10M_RANGE            13164
WD10M                  13164
ALLSKY_SFC_SW_DIFF     33877
WS50M                  13164
WS50M_MAX              13164
WS50M_MIN              13164
WS50M_RANGE            13164
WD50M                  13164
ALLSKY_KT              33877
ALLSKY_SFC_LW_DWN      32491
ALLSKY_SFC_PAR_TOT     33877
CLRSKY_SFC_PAR_TOT     33877
ALLSKY_SFC_UVA         33877
ALLSKY_SFC_UVB         33877
dtype: int64

In [138]:
nasa_data.interpolate(method="linear", limit_direction="both", inplace=True)
# nasa_data.fillna(method="bfill", inplace=True)
# nasa_data.fillna(method="ffill", inplace=True)
nasa_data.to_csv("data/nasa/merged/all_variables_merged_interpolated.csv", index=False)
missing_after_filtering = nasa_data.isnull().sum()
missing_after_filtering[missing_after_filtering > 0]

Series([], dtype: int64)

In [139]:
numeric_cols = nasa_data.select_dtypes(include=["float64", "int64"]).columns
z_scores = nasa_data[numeric_cols].apply(zscore)
nasa_data = nasa_data[(z_scores.abs() <= 3).all(axis=1)]
nasa_data.isnull().sum().sum()

np.int64(0)

Normalize selected features for AI model input

In [140]:
scaling_cols = [
    "ALLSKY_SFC_SW_DWN", "ALLSKY_SFC_SW_DNI", "ALLSKY_SFC_PAR_TOT", "CLRSKY_SFC_PAR_TOT", 
    "WS10M", "WS10M_MAX", "WS50M", "WS50M_MAX", "T2M", "RH2M", "PRECTOTCORR"
]

In [141]:
nasa_data[scaling_cols] = (nasa_data[scaling_cols] - nasa_data[scaling_cols].min()) / (
    nasa_data[scaling_cols].max() - nasa_data[scaling_cols].min()
)

In [142]:
missing_values_after = nasa_data.isnull().sum()


missing_data_summary_after = pd.DataFrame(
    {
        "Missing Values": missing_values_after,
        "Percentage": (missing_values_after / len(nasa_data)) * 100,
    }
).sort_values(by="Missing Values", ascending=False)

missing_data_summary_after.head()

,Missing Values,Percentage
LAT,0,0.0
LON,0,0.0
YEAR,0,0.0
MO,0,0.0
DY,0,0.0


In [143]:
nasa_data.head()

,LAT,LON,YEAR,MO,DY,ALLSKY_SFC_UV_INDEX,ALLSKY_SFC_SW_DWN,WS2M,T2M,T2MDEW,...,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB
26679,31.0,34.0,2024,1,1,0.49,0.864365,0.97,0.274439,7.8,...,0.214352,0.63,2.97,289.5,0.51,9.02,0.139683,0.158273,0.16,0.0
26680,31.0,34.0,2024,1,2,0.49,0.864365,0.97,0.274439,7.8,...,0.214352,0.63,2.97,289.5,0.51,9.02,0.139683,0.158273,0.16,0.0
26681,31.0,34.0,2024,1,3,0.49,0.864365,0.97,0.274439,7.8,...,0.214352,0.63,2.97,289.5,0.51,9.02,0.139683,0.158273,0.16,0.0
26682,31.0,34.0,2024,1,4,0.49,0.864365,0.97,0.274439,7.8,...,0.214352,0.63,2.97,289.5,0.51,9.02,0.139683,0.158273,0.16,0.0
26683,31.0,34.0,2024,1,5,0.49,0.864365,0.97,0.274439,7.8,...,0.214352,0.63,2.97,289.5,0.51,9.02,0.139683,0.158273,0.16,0.0


In [144]:
# if not exist
os.makedirs("outputs/exploring_outputs/nasa", exist_ok=True)
nasa_data.describe().to_csv("outputs/exploring_outputs/nasa/nasa_interpolated_description.csv", index=False)
nasa_data.describe()


,LAT,LON,YEAR,MO,DY,ALLSKY_SFC_UV_INDEX,ALLSKY_SFC_SW_DWN,WS2M,T2M,T2MDEW,...,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB
count,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,...,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000,36594.000000
mean,32.003621,35.003180,2022.319861,6.629147,15.688556,1.863871,0.558856,2.230834,0.473459,11.682553,...,0.464449,1.976985,4.306555,244.337127,0.625482,8.367288,0.526479,0.529418,0.312860,0.005293
std,0.681368,0.571481,1.485287,3.424966,8.821463,1.136931,0.242743,1.061150,0.191068,4.915016,...,0.139425,1.217475,1.407938,89.192724,0.093339,0.613277,0.313894,0.305809,0.125688,0.004297
min,31.000000,34.000000,2020.000000,1.000000,1.000000,0.270000,0.000000,0.280000,0.000000,-3.640000,...,0.000000,0.000000,0.830000,0.000000,0.340000,6.510000,0.000000,0.000000,0.100000,0.000000
25%,31.500000,34.375000,2021.000000,4.000000,8.000000,0.505880,0.334155,1.498904,0.307623,8.200663,...,0.360671,1.060000,3.290019,220.900000,0.510000,7.920000,0.146684,0.158273,0.160000,0.000000
50%,32.000000,35.000000,2022.000000,7.000000,16.000000,1.857458,0.571247,2.120000,0.420030,10.239612,...,0.452936,1.770000,4.102311,273.500000,0.640000,8.316506,0.533452,0.536454,0.316338,0.005753
75%,32.500000,35.625000,2024.000000,10.000000,23.000000,3.260000,0.803460,2.750000,0.654709,15.180000,...,0.552656,2.695413,5.210000,303.498842,0.730000,8.857685,0.898413,0.888489,0.460000,0.010000
max,33.000000,36.000000,2024.000000,12.000000,31.000000,3.440000,1.000000,6.110000,1.000000,24.330000,...,1.000000,6.510000,9.130000,360.000000,0.790000,10.180000,1.000000,1.000000,0.510000,0.010000


The dataset has been cleaned and prepared for the **`Renewable Energy Consumption Tracker`**. Key steps taken:

✅ Handled Missing Values: Replaced -999 with NaN and applied interpolation.

✅ Removed Duplicates: Dropped redundant columns.

✅ Filtered by Location: Kept only valid latitude/longitude values for Palestine.

✅ Outlier Detection & Removal: Used Z-score filtering to remove extreme values.

✅ Feature Normalization: Scaled key variables for AI model compatibility.


In [154]:
nasa_data.shape

(36594, 37)

In [155]:
nasa_data.isnull().sum().sum()


np.int64(0)

In [156]:
nasa_data.duplicated().sum()

np.int64(0)

In [157]:
nasa_data.dtypes

LAT                    float64
LON                    float64
YEAR                     int64
MO                       int64
DY                       int64
ALLSKY_SFC_UV_INDEX    float64
ALLSKY_SFC_SW_DWN      float64
WS2M                   float64
T2M                    float64
T2MDEW                 float64
T2MWET                 float64
TS                     float64
T2M_RANGE              float64
T2M_MAX                float64
ALLSKY_SFC_SW_DNI      float64
T2M_MIN                float64
QV2M                   float64
RH2M                   float64
PRECTOTCORR            float64
PS                     float64
WS10M                  float64
WS10M_MAX              float64
WS10M_MIN              float64
WS10M_RANGE            float64
WD10M                  float64
ALLSKY_SFC_SW_DIFF     float64
WS50M                  float64
WS50M_MAX              float64
WS50M_MIN              float64
WS50M_RANGE            float64
WD50M                  float64
ALLSKY_KT              float64
ALLSKY_S

Now the dataset is well-structured, free of missing values and duplicates, and correctly formatted for further analysis or machine learning models.

In [145]:
os.makedirs("outputs/preprocessed_data", exist_ok=True)
nasa_data.to_csv("outputs/preprocessed_data/nasa_data_cleaned.csv", index=False)

In [146]:
!jupyter nbconvert --to script "nasa_dataset_inspection.ipynb" --output-dir="outputs/scripts"
!jupyter nbconvert --to html "nasa_dataset_inspection.ipynb" --output-dir="outputs/html"

C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
[NbConvertApp] Converting notebook nasa_dataset_inspection.ipynb to script
[NbConvertApp] Writing 9658 bytes to outputs\scripts\nasa_dataset_inspection.py
C:\Users\obada\AppData\Local\Programs\Python\Python313\Scripts\jupyter-nbconvert.EXE\__main__.py:4: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a sp